Beyond the Model: Data Ethics

In [ ]:
import sqlite3
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from category_encoders import OneHotEncoder
from IPython.display import VimeoVideo
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.utils.validation import check_is_fitted

warnings.simplefilter(action="ignore", category=FutureWarning)

Prepare Data

In [ ]:
#Run the cell below to connect to the nepal.sqlite database
%load_ext sql
%sql sqlite:////home/jovyan/nepal.sqlite

In [ ]:
#Select all columns from the household_demographics table, limiting your results to the first five rows.
%%sql
SELECT *
FROM household_demographics
LIMIT 5

In [ ]:
#How many observations are in the household_demographics table? Use the count command to find out
%%sql
SELECT count(*)
FROM household_demographics
LIMIT 5

In [ ]:
#Select all columns from the id_map table, limiting your results to the first five rows
%%sql
SELECT *
FROM id_map
LIMIT 5

In [ ]:
#Create a table with all the columns from household_demographics, all the columns from building_structure, the vdcmun_id column from id_map, and the damage_grade column from building_damage. Your results should show only rows where the district_id is 4 and limit your results to the first five rows
%%sql
SELECT h.*,
        s.*,
        d.damage_grade,
        i.vdcmun_id
From household_demographics AS h 
JOIN id_map AS i ON i.household_id= h.household_id
JOIN building_structure AS s ON i.building_id=s.building_id
JOIN building_damage AS d ON i.building_id=d.building_id
WHERE district_id=4
LIMIT 5

Import

In [ ]:
def wrangle(db_path):
    # Connect to database
    conn = sqlite3.connect(db_path)

    # Construct query
    query = """SELECT h.*,
        s.*,
        d.damage_grade,
        i.vdcmun_id
From household_demographics AS h 
JOIN id_map AS i ON i.household_id= h.household_id
JOIN building_structure AS s ON i.building_id=s.building_id
JOIN building_damage AS d ON i.building_id=d.building_id
WHERE district_id=4"""

    # Read query results into DataFrame
    df = pd.read_sql(query, conn, index_col= "household_id")

    # Identify leaky columns
    drop_cols = [col for col in df.columns if "post_eq" in col]

    # Add high-cardinality / redundant column
    drop_cols.append("building_id")

    # Create binary target column
    df["damage_grade"] = df["damage_grade"].str[-1].astype(int)
    df["severe_damage"] = (df["damage_grade"] > 3).astype(int)

    # Drop old target
    drop_cols.append("damage_grade")

    # Drop multicollinearity column
    drop_cols.append("count_floors_pre_eq")
    
    # Drop columns
    df.drop(columns=drop_cols, inplace=True)

    return df

In [ ]:
df = wrangle("/home/jovyan/nepal.sqlite")
df.head()

In [ ]:
#Combine the select_dtypes and nunique methods to see if there are any high- or low-cardinality categorical features in the dataset
# Check for high- and low-cardinality categorical features
df.select_dtypes("object").nunique()

In [ ]:
df["caste_household"].value_counts().head(10)

In [ ]:
df["caste_household"].value_counts().tail(10)

In [ ]:
df["caste_household"].value_counts().head(10).index

In [ ]:
top_10=df["caste_household"].value_counts().head(10).index
top_10

In [ ]:
#Add to your wrangle function so that the "caste_household" contains only the 10 largest caste groups. For the rows that are not in those groups, "caste_household" should be changed to "Other"
df["caste_household"].apply(lambda c : c if c in top_10 else "other").value_counts()

In [ ]:
def wrangle(db_path):
    # Connect to database
    conn = sqlite3.connect(db_path)

    # Construct query
    query = """SELECT h.*,
        s.*,
        d.damage_grade,
        i.vdcmun_id
From household_demographics AS h 
JOIN id_map AS i ON i.household_id= h.household_id
JOIN building_structure AS s ON i.building_id=s.building_id
JOIN building_damage AS d ON i.building_id=d.building_id
WHERE district_id=4"""

    # Read query results into DataFrame
    df = pd.read_sql(query, conn, index_col= "household_id")

    # Identify leaky columns
    drop_cols = [col for col in df.columns if "post_eq" in col]

    # Add high-cardinality / redundant column
    drop_cols.append("building_id")

    # Create binary target column
    df["damage_grade"] = df["damage_grade"].str[-1].astype(int)
    df["severe_damage"] = (df["damage_grade"] > 3).astype(int)

    # Drop old target
    drop_cols.append("damage_grade")

    # Drop multicollinearity column
    drop_cols.append("count_floors_pre_eq")
    
    #Group caste column 
    top_10=df["caste_household"].value_counts().head(10).index
    df["caste_household"]=df["caste_household"].apply(
        lambda c : c if c in top_10 else "other"
    )
    
    # Drop columns
    df.drop(columns=drop_cols, inplace=True)

    return df

In [ ]:
df = wrangle("/home/jovyan/nepal.sqlite")
df.head()

Split

In [ ]:
#Create your feature matrix X and target vector y. Since our model will only consider building and household data, X should not include the municipality column "vdcmun_id". Your target is "severe_damage"
target = "severe_damage"
X = df.drop(columns = [target, "vdcmun_id"])
y = df[target]

In [ ]:
#Divide your data (X and y) into training and test sets using a randomized train-test split. Your test set should be 20% of your total data. Be sure to set a random_state for reproducibility.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42

In [ ]:
#Calculate the baseline accuracy score for your model
acc_baseline =y_train.value_counts(normalize=True).max()
print("Baseline Accuracy:", round(acc_baseline, 2))

In [ ]:
#Create a Pipeline called model_lr. It should have an OneHotEncoder transformer and a LogisticRegression predictor. Be sure you set the use_cat_names argument for your transformer to True
model_lr = make_pipeline(
    OneHotEncoder(use_cat_names=True), LogisticRegression(max_iter=3000) 
)
model_lr.fit(X_train, y_train)

In [ ]:
#Calculate the training and test accuracy scores for model_lr
acc_train = accuracy_score(y_train, model_lr.predict(X_train))
acc_test = model_lr.score(X_test, y_test)

print("LR Training Accuracy:", acc_train)
print("LR Validation Accuracy:", acc_test)

In [ ]:
#First, extract the feature names and importances from your model. Then create a pandas Series named feat_imp, where the index is features and the values are your the exponential of the importances
features = model_lr.named_steps["onehotencoder"].get_feature_names()
importances = model_lr.named_steps["logisticregression"].coef_[0]
feat_imp = pd.Series(np.exp(importances), index=features).sort_values()
feat_imp.head()

In [ ]:
#Create a horizontal bar chart with the ten largest coefficients from feat_imp. Be sure to label your x-axis "Odds Ratio"
feat_imp.tail(10).plot(kind="barh")
plt.xlabel("Odds Ratio")

In [ ]:
#Create a horizontal bar chart with the ten smallest coefficients from feat_imp. Be sure to label your x-axis "Odds Ratio"
feat_imp.head(10).plot(kind="barh")
plt.xlabel("Odds Ratio")

In [ ]:
#Which municipalities saw the highest proportion of severely damaged buildings? Create a DataFrame damage_by_vdcmun by grouping df by "vdcmun_id" and then calculating the mean of the "severe_damage" column. Be sure to sort damage_by_vdcmun from highest to lowest proportion
damage_by_vdcmun = (
    df.groupby("vdcmun_id")["severe_damage"].mean().sort_values(ascending= False)
).to_frame()
damage_by_vdcmun

In [ ]:
(
    df[df["caste_household"]=="Gurung"].groupby("vdcmun_id")["severe_damage"].count()/df.groupby("vdcmun_id")["severe_damage"].count()
)

In [ ]:
#Create a new column in damage_by_vdcmun that contains the the proportion of Gurung households in each municipality
damage_by_vdcmun["Gurung"] = (
    df[df["caste_household"]=="Gurung"].groupby("vdcmun_id")["severe_damage"].count()/df.groupby("vdcmun_id")["severe_damage"].count()
)

In [ ]:
#Create a new column in damage_by_vdcmun that contains the the proportion of Kumal households in each municipality. Replace any NaN values in the column with 0
damage_by_vdcmun["Kumal"] =  (
    df[df["caste_household"]=="Kumal"].groupby("vdcmun_id")["severe_damage"].count()/df.groupby("vdcmun_id")["severe_damage"].count()
).fillna(0)